In [9]:
# dependencies
import re
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import string
import nltk
nltk.download('averaged_perceptron_tagger');

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, PCA
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from catboost import Pool, CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import log_loss
from imblearn.over_sampling import SMOTE, RandomOverSampler

import warnings
warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams["figure.figsize"] = (12, 8)
pd.set_option('display.max_columns', None)

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>


In [128]:
Train = pd.read_csv('Train.csv')
Test = pd.read_csv('Test.csv')
extra = pd.read_csv('extra_data.csv')
sub = pd.read_csv('SampleSubmission.csv')
random_seed = 12 # random seed for all computations
Train.shape, Test.shape, extra.shape, sub.shape

((373, 12), (558, 11), (10000, 12), (558, 14))

In [129]:
data = pd.concat([Train, Test], sort = False).reset_index(drop = True)
data = data.sort_values('PURCHASED_AT').reset_index(drop = True) 

In [130]:
# def get_isholiday(date):
#     holiday_list = ['01-01-21', '14-02-21', '28-08-21', '14-04-21', '16-04-21', '21-04-21',
#                     '01-05-21', '15-06-21', '07-09-21', '12-10-21', '02-11-21', '15-11-21', 
#                     '24-12-21', '25-12-21', '31-12-21',
#                     '01-01-22', '14-02-22', '28-08-22', '14-04-22', '16-04-22', '21-04-22',
#                     '01-05-22', '15-06-22', '07-09-22', '12-10-22', '02-11-22', '15-11-22', 
#                     '24-12-22', '25-12-22', '31-12-22']
#     date = date.strftime(format='%d-%m-%y') 
#     if date in holiday_list:
#         return 1
#     else:
#         return 0

data['created_at_1'] = pd.to_datetime(data['MERCHANT_CATEGORIZED_AT'])
data['created_at'] = pd.to_datetime(data['PURCHASED_AT'])
# last_day = data.created_at.max()
data['year'] = data['created_at'].dt.year
data['month'] = data['created_at'].dt.month
data['day'] = data['created_at'].dt.day
# data['hour'] = data['created_at'].dt.hour
# data['minute'] = data['created_at'].dt.minute
# data['second'] = data['created_at'].dt.second
# data['week'] = data['created_at'].dt.week
# data['dayofweek'] = data['created_at'].dt.dayofweek
# data['weekofyear'] = data['created_at'].dt.weekofyear
# data['ismonthend'] = data['created_at'].dt.is_month_end
# data['ismonthstart'] = data['created_at'].dt.is_month_start
# data['quarter'] = data.created_at.dt.quarter
# data['period'] = pd.cut(data['hour'], bins = [0, 4, 8, 12, 16, 20, 23], labels = ['Midnight', 'Early_Morning', 'Late_Morning', 'Afternoon', 'Evening', 'Night'])
# data['year_month'] = data['year'].astype(str) + '-' + data['month'].astype(str)
# data['month_day'] = data['month'].astype(str) + '-' + data['day'].astype(str)
data['datediff'] = (data['created_at'] - data['created_at_1']).dt.total_seconds() / (60 * 60 * 24)
# data['month_from_today'] = (12 * (last_day.year - data['created_at'].dt.year) + (last_day.month - data['created_at'].dt.month))
# data['month_1'] = (last_day.day - data['created_at_1'].dt.day)//30
# data['categorized_on_holiday'] = data['created_at'].apply(lambda date_obj: get_isholiday(date_obj))

In [131]:
# data_pc = data.copy()
data_ex = pd.concat([Train, Test, extra], sort = False).reset_index(drop = True)
def Agg_1(Feature):
    for key in ('USER_ID','MERCHANT_NAME','IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY'):
        data[f'{Feature}_{key}_Mean'] = data[key].map(dict(data_ex.groupby(key)[Feature].mean()))
        # data[f'{Feature}_{key}_Quantile10'] = data[key].map(dict(data_ex.groupby(key)[Feature].quantile(0.10)))
        data[f'{Feature}_{key}_Quantile25'] = data[key].map(dict(data_ex.groupby(key)[Feature].quantile(0.25)))
        data[f'{Feature}_{key}_Quantile75'] = data[key].map(dict(data_ex.groupby(key)[Feature].quantile(0.75)))
        # data[f'{Feature}_{key}_Quantile90'] = data[key].map(dict(data_ex.groupby(key)[Feature].quantile(0.90)))
        data[f'{Feature}_{key}_Std'] = data[key].map(dict(data_ex.groupby(key)[Feature].std()))
        # data[f'{Feature}_{key}_Min'] = data[key].map(dict(data_ex.groupby(key)[Feature].min()))
        # data[f'{Feature}_{key}_Max'] = data[key].map(dict(data_ex.groupby(key)[Feature].max()))
        data[f'{Feature}_{key}_Sum'] = data[key].map(dict(data_ex.groupby(key)[Feature].sum()))
        # data[f'{Feature}_{key}_Range'] = data[key].map(dict(data_ex.groupby(key)[Feature].agg(np.ptp)))
        # data[f'{Feature}_{key}_Kurt'] = data[key].map(dict(data_ex.groupby(key)[Feature].agg(pd.Series.kurt)))
        data[f'{Feature}_{key}_Skew'] = data[key].map(dict(data_ex.groupby(key)[Feature].agg(pd.Series.skew)))
        # data[f'{Feature}_{key}_Count'] = data[key].map(dict(data_ex.groupby(key)[Feature].count()))
        data[f'{Feature}_{key}_NUnique'] = data[key].map(dict(data_ex.groupby(key)[Feature].nunique()))
        # data[f'{Feature}_{key}_Mode'] = data[key].map(dict(data_ex.groupby(key)[Feature].apply(lambda x: x.value_counts().keys()[0])))
        
Agg_1('PURCHASE_VALUE')

In [132]:
for col in data.columns.difference(['MERCHANT_CATEGORIZED_AS']):
    if data[col].isna().sum() > 0:
        data[col].fillna(data[col].ffill().bfill(), inplace = True)
# data.isnull().sum()

In [133]:
# data.columns.to_list()

In [134]:
# data_dec = data_pc[['PURCHASE_VALUE_USER_ID_Mean',
#  'PURCHASE_VALUE_USER_ID_Quantile10',
#  'PURCHASE_VALUE_USER_ID_Quantile25',
#  'PURCHASE_VALUE_USER_ID_Quantile75',
#  'PURCHASE_VALUE_USER_ID_Quantile90',
#  'PURCHASE_VALUE_USER_ID_Std',
#  'PURCHASE_VALUE_USER_ID_Min',
#  'PURCHASE_VALUE_USER_ID_Max',
#  'PURCHASE_VALUE_USER_ID_Sum',
#  'PURCHASE_VALUE_USER_ID_Range',
#  'PURCHASE_VALUE_USER_ID_Kurt',
#  'PURCHASE_VALUE_USER_ID_Skew',
#  'PURCHASE_VALUE_USER_ID_Count',
#  'PURCHASE_VALUE_USER_ID_NUnique',
#  'PURCHASE_VALUE_USER_ID_Mode',
#  'PURCHASE_VALUE_MERCHANT_NAME_Mean',
#  'PURCHASE_VALUE_MERCHANT_NAME_Quantile10',
#  'PURCHASE_VALUE_MERCHANT_NAME_Quantile25',
#  'PURCHASE_VALUE_MERCHANT_NAME_Quantile75',
#  'PURCHASE_VALUE_MERCHANT_NAME_Quantile90',
#  'PURCHASE_VALUE_MERCHANT_NAME_Std',
#  'PURCHASE_VALUE_MERCHANT_NAME_Min',
#  'PURCHASE_VALUE_MERCHANT_NAME_Max',
#  'PURCHASE_VALUE_MERCHANT_NAME_Sum',
#  'PURCHASE_VALUE_MERCHANT_NAME_Range',
#  'PURCHASE_VALUE_MERCHANT_NAME_Kurt',
#  'PURCHASE_VALUE_MERCHANT_NAME_Skew',
#  'PURCHASE_VALUE_MERCHANT_NAME_Count',
#  'PURCHASE_VALUE_MERCHANT_NAME_NUnique',
#  'PURCHASE_VALUE_MERCHANT_NAME_Mode',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Mean',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Quantile10',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Quantile25',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Quantile75',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Quantile90',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Std',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Min',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Max',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Sum',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Range',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Kurt',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Skew',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Count',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_NUnique',
#  'PURCHASE_VALUE_IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_Mode',
#  'PURCHASE_VALUE_period_Mean',
#  'PURCHASE_VALUE_period_Quantile10',
#  'PURCHASE_VALUE_period_Quantile25',
#  'PURCHASE_VALUE_period_Quantile75',
#  'PURCHASE_VALUE_period_Quantile90',
#  'PURCHASE_VALUE_period_Std',
#  'PURCHASE_VALUE_period_Min',
#  'PURCHASE_VALUE_period_Max',
#  'PURCHASE_VALUE_period_Sum',
#  'PURCHASE_VALUE_period_Range',
#  'PURCHASE_VALUE_period_Kurt',
#  'PURCHASE_VALUE_period_Skew',
#  'PURCHASE_VALUE_period_Count',
#  'PURCHASE_VALUE_period_NUnique',
#  'PURCHASE_VALUE_period_Mode']]

# for col in data_dec.columns:
#     data_dec[col].fillna(data_dec[col].ffill().bfill(), inplace = True)
# data_dec.isnull().sum()

In [135]:
# # decomposition using PCA
# pca = PCA(random_state = random_seed, n_components = 3)
# data_pca = pca.fit_transform(data_dec)
# data['PC_1'] = 0
# # data['PC_2'] = 0
# # data['PC_3'] = 0
# data['PC_1'] = data_pca

In [138]:
data.USER_GENDER.fillna('Female', inplace = True)
# data.USER_AGE.fillna(25.0, inplace = True)
# data.period.fillna('Afternoon', inplace = True)
# values = data.mode().loc[0]
# data = data.fillna(values)

le = LabelEncoder()
LE_cols = ['IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY', 'USER_ID', 'USER_GENDER']
for le_col in LE_cols:
    data[le_col] = le.fit_transform(data[le_col])

data.drop(['Transaction_ID', 'created_at', 'created_at_1', 'MERCHANT_CATEGORIZED_AT', 'PURCHASED_AT', 'USER_AGE'], 1, inplace = True)

In [139]:
CouVec = CountVectorizer()
CouVec.fit(data['MERCHANT_NAME'])

data_words = pd.DataFrame(CouVec.transform(data['MERCHANT_NAME']).toarray())

In [140]:
def tag_part_of_speech(text):
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    pos_list = nltk.pos_tag(text_splited)
    noun_count = len([w for w in pos_list if w[1] in ('NN','NNP','NNPS','NNS')])
    adjective_count = len([w for w in pos_list if w[1] in ('JJ','JJR','JJS')])
    verb_count = len([w for w in pos_list if w[1] in ('VB','VBD','VBG','VBN','VBP','VBZ')])
    return[noun_count, adjective_count, verb_count]
  
data['Merchant_Name_Length'] = data['MERCHANT_NAME'].apply(lambda x : len(x))
data['Num_Words'] = data['MERCHANT_NAME'].apply(lambda comment: len(comment.split()))
data['Num_Unique_Words'] = data['MERCHANT_NAME'].apply(lambda comment: len(set(w for w in comment.split())))
data['Words_VS_Unique'] = data['Num_Unique_Words'] / data['Num_Words']
data['num_punctuation'] = data['MERCHANT_NAME'].apply(lambda comment: sum(comment.count(w) for w in '\'.,;:'))
data['nouns'], data['adjectives'], data['verbs'] = zip(*data['MERCHANT_NAME'].apply(lambda comment: tag_part_of_speech(comment)))
data['nouns_vs_length'] = data['nouns'] / data['Merchant_Name_Length']
data['adjectives_vs_length'] = data['adjectives'] / data['Merchant_Name_Length']
data['verbs_vs_length'] = data['verbs'] /data['Merchant_Name_Length']
data['nouns_vs_words'] = data['nouns'] / data['Num_Words']
data['adjectives_vs_words'] = data['adjectives'] / data['Num_Words']
data['verbs_vs_words'] = data['verbs'] / data['Num_Words']
data["count_words_title"] = data["MERCHANT_NAME"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
data["mean_word_len"] = data["MERCHANT_NAME"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
data['punct_percent']= data['num_punctuation']*100 / data['Num_Words']

In [141]:
LDA = LatentDirichletAllocation(n_components=15, max_iter=100, random_state=42)
LDA.fit(data_words)
Topics = [f'Topic_{x}' for x in range(0,15)]
data[Topics] = LDA.transform(data_words)

In [142]:
# n_top_words = 10
# topic_summaries = []

# # get topics and topic terms
# topic_word = LDA.components_ 
# vocab = CouVec.get_feature_names()

# for i, topic_dist in enumerate(topic_word):
#     topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
#     topic_summaries.append(' '.join(topic_words))
#     print('Topic {}: {}'.format(i, ' | '.join(topic_words)))

In [143]:
# for a, b in Train.MERCHANT_NAME.groupby([Train.MERCHANT_CATEGORIZED_AS]):
#     print(a, b)

In [144]:
data['MERCHANT_NAME'] = le.fit_transform(data['MERCHANT_NAME'])

In [145]:
# separating data into train and test
train = data[data.MERCHANT_CATEGORIZED_AS.notnull()].reset_index(drop = True)
test = data[data.MERCHANT_CATEGORIZED_AS.isna()].reset_index(drop = True)
test.drop('MERCHANT_CATEGORIZED_AS', axis = 1, inplace = True)
print('shape', train.shape, test.shape)

shape (373, 65) (558, 64)


In [146]:
train['MERCHANT_CATEGORIZED_AS'] = le.fit_transform(train['MERCHANT_CATEGORIZED_AS'].values)

In [147]:
# plt.figure(figsize = (15, 15))
# sns.heatmap(data.corr(), cmap = 'coolwarm', square = True)
# plt.show()

In [148]:
#Duplicated
cols = train.columns
dup = []
for feat_1 in cols:
    if (feat_1 in dup):
        continue
    for feat_2 in cols.drop(feat_1):
        if (feat_2 in dup):
            continue
        if (train[feat_1].equals(train[feat_2])):
            train.drop(feat_2,inplace=True,axis=1)
            test.drop(feat_2,inplace=True,axis=1)
            dup.append(feat_2)

In [149]:
#Constant
for feat in test.columns:
    if ((len(train[feat].value_counts().keys()) == 1) | (len(test[feat].value_counts().keys()) == 1)):
        train.drop(feat,inplace=True,axis=1)
        test.drop(feat,inplace=True,axis=1)

In [264]:
cb_params = {'iterations' : 1300, 'learning_rate' : 0.09, 'depth' : 5, 'reg_lambda' : 17.56889442852513, 'verbose' : 0}
cb = CatBoostClassifier(**cb_params, random_state = random_seed)

rf_params = {'n_estimators': 4600, 'max_depth': 12, 'min_samples_split': 3,'min_samples_leaf': 1, 'max_features': 0.25}
rf_ = RandomForestClassifier(**rf_params, n_jobs= -1, random_state = random_seed)

xgb_ = XGBClassifier(random_state = random_seed, objective = 'multi:softprob', num_class = 13, learning_rate = 0.05976,  max_depth = 6, 
                    subsample = 0.5, colsample_bytree = 0.5, eval_metric='mlogloss')

In [267]:
print('Validating...')
X_split = train.drop('MERCHANT_CATEGORIZED_AS',axis = 1).values
y_split = train['MERCHANT_CATEGORIZED_AS'].values

scores = []
for train_index, test_index in StratifiedKFold(n_splits = 2, shuffle = True, random_state = random_seed).split(X_split, y_split):
    X_Train, X_Test = X_split[train_index], X_split[test_index]
    y_Train, y_Test = y_split[train_index], y_split[test_index]
    cb.fit(X_Train, y_Train)
    rf_.fit(X_Train, y_Train)
    xgb_.fit(X_Train, y_Train)

    y_Pred = 0.15 * xgb_.predict_proba(X_Test) + 0.45 * cb.predict_proba(X_Test) + 0.4 * rf_.predict_proba(X_Test)
    scores.append(log_loss(y_Test, y_Pred))
    print(log_loss(y_Test, y_Pred))

print("\nMean:",np.mean(scores),"\nSTD: ", np.std(scores))

Validating...
1.4797753137656173
1.5759520525947504

Mean: 1.527863683180184 
STD:  0.04808836941456651


In [270]:
print('Validating...')
X_split = train.drop('MERCHANT_CATEGORIZED_AS',axis=1).values
y_split = train['MERCHANT_CATEGORIZED_AS'].values

scores = []
for train_index, test_index in KFold(n_splits = 2, shuffle = True, random_state = random_seed).split(X_split, y_split):
    X_Train, X_Test = X_split[train_index], X_split[test_index]
    y_Train, y_Test = y_split[train_index], y_split[test_index]
    cb.fit(X_Train, y_Train)
    rf_.fit(X_Train, y_Train)
    xgb_.fit(X_Train, y_Train)

    y_Pred = 0.2 * xgb_.predict_proba(X_Test) + 0.47 * cb.predict_proba(X_Test) + 0.33 * rf_.predict_proba(X_Test)
    scores.append(log_loss(y_Test, y_Pred))
    print(scores[-1])

print("\nMean:",np.mean(scores),"\nSTD: ", np.std(scores))

Validating...
1.4638703907315687
1.6101444209907403

Mean: 1.5370074058611545 
STD:  0.07313701512958581


In [253]:
# use_feat = ['MERCHANT_NAME', 'PURCHASE_VALUE', 'IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY', 'USER_HOUSEHOLD', 'USER_INCOME', 'delta_days', 'year', 'month', 'cluster', 'merchant_dec']
# X_, y_ = train[use_feat], train["MERCHANT_CATEGORIZED_AS"]
X_, y_ = train.drop("MERCHANT_CATEGORIZED_AS", axis = 1), train["MERCHANT_CATEGORIZED_AS"]
X, y = SMOTE(k_neighbors = 1, random_state = 1).fit_resample(X_, y_)
skf = StratifiedKFold(n_splits = 5, random_state = random_seed, shuffle = True)

In [254]:
cat_scores = []
for fold,(tr_in,te_in) in enumerate(skf.split(X, y)):
    print("===== Fold {fold} =====".format(fold = fold + 1))
    X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
    y_train, y_test = y.iloc[tr_in], y.iloc[te_in]
    cat = CatBoostClassifier(iterations = 1300, learning_rate = 0.09, depth = 6, reg_lambda = 17.56889442852513, verbose = 0, random_state = random_seed)
    cat.fit(X_train, y_train, eval_set = [(X_train, y_train),(X_test, y_test)], early_stopping_rounds = 500, use_best_model = True)
    loss = log_loss(y_test, cat.predict_proba(X_test))
    cat_scores.append(loss)
    print(loss)

print('cat_score : ' + str(np.mean(cat_scores)))

===== Fold 1 =====
0.5855852298197676
===== Fold 2 =====
0.42631013867016726
===== Fold 3 =====
0.4829029132561286
===== Fold 4 =====
0.38934389371334926
===== Fold 5 =====
0.5328382556507935
cat_score : 0.4833960862220413


In [259]:
rf_scores = []
for fold,(tr_in,te_in) in enumerate(skf.split(X, y)):
    print("===== Fold {fold} =====".format(fold = fold + 1))
    X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
    y_train, y_test = y.iloc[tr_in], y.iloc[te_in]
    rf = RandomForestClassifier(n_estimators = 4600, max_depth = 12, min_samples_split = 3, min_samples_leaf = 1, max_features = 0.25, n_jobs= -1, random_state = random_seed)
    rf.fit(X_train, y_train)
    loss = log_loss(y_test, rf.predict_proba(X_test))
    rf_scores.append(loss)
    print(loss)

print('\nrf_score ' + str(np.mean(rf_scores)))

===== Fold 1 =====
0.6200538923137134
===== Fold 2 =====
0.4726855188457359
===== Fold 3 =====
0.48842859128751726
===== Fold 4 =====
0.4310253878233724
===== Fold 5 =====
0.5877848570619839

rf_score 0.5199956494664646


In [261]:
xgb_scores = []
for fold,(tr_in,te_in) in enumerate(skf.split(X, y)):
    print("===== Fold {fold} =====".format(fold = fold + 1))
    X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
    y_train, y_test = y[tr_in], y[te_in]
    xgb = XGBClassifier(random_state = random_seed, objective = 'multi:softprob', num_class = 13, learning_rate = 0.05976,  max_depth = 6, 
                        subsample = 0.5, colsample_bytree = 0.5, eval_metric='mlogloss')
    xgb.fit(X_train, y_train)
    loss = log_loss(y_test, xgb.predict_proba(X_test))
    xgb_scores.append(loss)
    print(loss)

print('\nxgb_score ' + str(np.mean(xgb_scores)))

===== Fold 1 =====
0.6350299193950489
===== Fold 2 =====
0.4776210954385291
===== Fold 3 =====
0.481356908966388
===== Fold 4 =====
0.4177657750715029
===== Fold 5 =====
0.5601506697262941

xgb_score 0.5143848737195527


In [276]:
vcl_scores = []
for fold,(tr_in,te_in) in enumerate(skf.split(X, y)):
    print("===== Fold {fold} =====".format(fold = fold + 1))
    X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
    y_train, y_test = y.iloc[tr_in], y.iloc[te_in]
    vcl = VotingClassifier([('cat', cat), ('rf', rf), ('xgb', xgb)], weights = (1, 1, 1), voting = 'soft')
    vcl.fit(X_train, y_train)
    loss = log_loss(y_test, vcl.predict_proba(X_test))
    vcl_scores.append(loss)
    print(loss)

print('\nvcl_score ' + str(np.mean(vcl_scores)))

===== Fold 1 =====
0.591244023824
===== Fold 2 =====
0.44579967223701605
===== Fold 3 =====
0.47114409414467917
===== Fold 4 =====
0.40003743660797286
===== Fold 5 =====
0.5409934702176725

vcl_score 0.48984373940626813


In [28]:
# feat_imp_df = pd.DataFrame(lgb.feature_importances_, columns = ['Importance'])
# feat_imp_df['Features'] = X_.columns
# feat_imp_df.sort_values(by = 'Importance', ascending = False)
# # print(feat_imp_df[feat_imp_df.Importance < 1].Features.to_list(), end = " ")

In [277]:
def predict_and_submit(test_, filename):
    d = {"Transaction_ID": sub["Transaction_ID"], 'Bills & Fees':test_[:, 0], 'Data & WiFi':test_[:, 1], 'Education':test_[:, 2], 'Emergency fund':test_[:, 3],'Family & Friends':test_[:, 4],'Going out':test_[:, 5],'Groceries':test_[:, 6],\
        'Health':test_[:, 7],'Loan Repayment':test_[:, 8],'Miscellaneous':test_[:, 9],'Rent / Mortgage':test_[:, 10],'Shopping':test_[:, 11],'Transport & Fuel':test_[:, 12]}
    df_ = pd.DataFrame(data=d)
    df_ = df_[["Transaction_ID", 'Bills & Fees','Data & WiFi','Education','Emergency fund','Family & Friends','Going out','Groceries','Health','Loan Repayment','Miscellaneous','Rent / Mortgage','Shopping','Transport & Fuel']]
    df_.to_csv(f'{filename}.csv', index = False)
    return df_.shape

In [278]:
y_a = cat.predict_proba(test)
y_b = rf.predict_proba(test)
y_c = xgb.predict_proba(test)
y_d = vcl.predict_proba(test)

In [279]:
pred = y_a * 0.3 + y_b * 0.2 + y_c * 0.1 + y_d * 0.4

In [280]:
predict_and_submit(y_e, 'Alvin_vcl_')

(558, 14)

In [282]:
Target_Values = Train['MERCHANT_CATEGORIZED_AS'].value_counts().keys().sort_values()
Target_Values

Index(['Bills & Fees', 'Data & WiFi', 'Education', 'Emergency fund',
       'Family & Friends', 'Going out', 'Groceries', 'Health',
       'Loan Repayment', 'Miscellaneous', 'Rent / Mortgage', 'Shopping',
       'Transport & Fuel'],
      dtype='object')

In [283]:
vcl_ = VotingClassifier([('cb', cb), ('rf_', rf_), ('xgb_', xgb_)], weights = (1, 1, 1), voting = 'soft')

In [285]:
X = train.drop('MERCHANT_CATEGORIZED_AS',axis=1)
y = train['MERCHANT_CATEGORIZED_AS']
        
cb.fit(X, y)
rf_.fit(X, y)
xgb_.fit(X, y)
vcl_.fit(X, y)

VotingClassifier(estimators=[('cb',
                              <catboost.core.CatBoostClassifier object at 0x0000021462E60B80>),
                             ('rf_',
                              RandomForestClassifier(max_depth=12,
                                                     max_features=0.25,
                                                     min_samples_split=3,
                                                     n_estimators=4600,
                                                     n_jobs=-1,
                                                     random_state=12)),
                             ('xgb_',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            callbacks=None, colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0...
                                            importance_type=None,
                                            interaction_constraints='',
                                            learning_rate=0.05976, max_bin=256,
                                            max_cat_to_onehot=4,
                                            max_delta_step=0, max_depth=6,
                                            max_leaves=0, min_child_weight=1,
                                            missing=nan,
                                            monotone_constraints='()',
                                            n_estimators=100, n_jobs=0,
                                            num_class=13, num_parallel_tree=1,
                                            objective='multi:softprob',
                                            predictor='auto', random_state=12, ...))],
                 voting='soft', weights=(1, 1, 1))

In [296]:
Predictions = vcl_.predict_proba(test)
ID = sub['Transaction_ID']
submission = pd.DataFrame({"Id": ID})
Predictions = pd.DataFrame(Predictions, index = test.index, columns = Target_Values)
submission = pd.concat([submission.reset_index(drop=True),Predictions.reset_index(drop=True)],axis=1)

In [297]:
submission.to_csv('aisquad0.csv', index=False)